In [2]:
!pip install jupyter
!pip install matplotlib
!pip install numpy
!pip install pandas
!pip install plotly
!pip install scipy
!pip install tqdm
!pip install Brian2

   ---------------------------------------- 0.0/5.0 MB ? eta -:--:--
    --------------------------------------- 0.1/5.0 MB 3.3 MB/s eta 0:00:02
   --- ------------------------------------ 0.5/5.0 MB 5.2 MB/s eta 0:00:01
   ------- -------------------------------- 0.9/5.0 MB 7.4 MB/s eta 0:00:01
   ------------- -------------------------- 1.7/5.0 MB 10.0 MB/s eta 0:00:01
   ---------------------- ----------------- 2.8/5.0 MB 12.7 MB/s eta 0:00:01
   --------------------------- ------------ 3.5/5.0 MB 13.1 MB/s eta 0:00:01
   -------------------------------- ------- 4.1/5.0 MB 13.2 MB/s eta 0:00:01
   -------------------------------------- - 4.8/5.0 MB 13.3 MB/s eta 0:00:01
   ---------------------------------------  5.0/5.0 MB 12.8 MB/s eta 0:00:01
   ---------------------------------------- 5.0/5.0 MB 11.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/123.4 kB ? eta -:--:--
   ------------------------------------ --- 112.6/123.4 kB 3.2 MB/s eta 0:00:01
   ----------

In [3]:
import os
from brian2 import *
import random
from tqdm.notebook import tqdm
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from plots import *
from equations import *
from global_settings import *
from masks import *
from helper import *
from run_loop import *

import time

In [4]:
def create_neuron_topology (N, bounds):
    
    x_bound, y_bound, z_bound = bounds
    
    x = [random.uniform(0, x_bound) for _ in range(N)]
    y = [random.uniform(0, y_bound) for _ in range(N)]
    z = [random.uniform(0, z_bound) for _ in range(N)]
    topology = np.array([x, y, z])
    
    return topology

In [5]:
def create_group_py(topology, noise, masks, group_name='exc_group', integ_method='exponential_euler'):

    x, y, z = topology
    mu_noise, sigma_noise = noise
    stimulus_mask, treatment_mask = masks
        
    G_exc=NeuronGroup(len(x),py_eqs,threshold='v>V_th',reset=reset_eqs,refractory=3*ms,name=group_name, method=integ_method)
    G_exc.v = '-60*mvolt-rand()*40*mvolt'
    G_exc.glu = 1
    G_exc.x = x * um
    G_exc.y = y * um
    G_exc.z = z * um

    G_exc.taille=taille_exc_normale

    G_exc.mu_noise = mu_noise
    G_exc.sigma_noise = sigma_noise

    G_exc.treatment_mask = treatment_mask
    G_exc.stimulus_mask = stimulus_mask

    return G_exc

In [6]:
def create_group_inh(topology, noise, treatment_mask, group_name='inh_group', integ_method='exponential_euler'):

    x, y,z = topology
    mu_noise, sigma_noise = noise
        
    G_inh=NeuronGroup(len(x),inh_eqs,threshold='v>V_th', name=group_name, refractory=3*ms,method=integ_method)
    G_inh.v = -60*mvolt-rand()*10*mvolt

    G_inh.taille=taille_inh_normale
    G_inh.x = x * um
    G_inh.y = y * um
    G_inh.z = z * um

    
    G_inh.mu_noise = mu_noise
    G_inh.sigma_noise = sigma_noise

    G_inh.treatment_mask = treatment_mask

    return G_inh

In [7]:
def create_group_lfp (bounds):

    x_bound, y_bound, z_bound = bounds
    
    # Set up a singular LFP electrode
    Ne = 1
    sigma = 0.3*siemens/meter # Resistivity of extracellular field (0.3-0.4 S/m)
    lfp = NeuronGroup(Ne, model='''v : volt
                                   x : meter
                                   y : meter
                                   z : meter''')
    lfp.x = x_bound/2*um
    lfp.y = y_bound/2*um
    lfp.z = z_bound/2*um

    return lfp

In [8]:
def read_input_signal (file_name):
    in_1 = np.loadtxt('./stimuli/'+file_name)

    input_signal = TimedArray(in_1*namp,dt=defaultclock.dt)

    return input_signal


In [9]:
def prepare_network (topologies, stimulus_mask_exc, treatment_masks, current_variables):

    # Expand parameter lists
    p_e2e, p_e2i, p_i2e, p_i2i = current_variables['p']
    topology_exc, topology_inh = topologies
    treatment_mask_exc, treatment_mask_inh = treatment_masks
    
    # Create py and inh groups
    G_exc = create_group_py(topology_exc, current_variables['noise_exc'], [stimulus_mask_exc, treatment_mask_exc])
    G_inh = create_group_inh(topology_inh, current_variables['noise_inh'], treatment_mask_inh)
    G_lfp = create_group_lfp(current_variables['bounds'])
    
    # Set up monitors
    popmon_exc = PopulationRateMonitor(G_exc)
    popmon_inh = PopulationRateMonitor(G_inh)
    Mlfp = StateMonitor(G_lfp, 'v', record=True)
    statemon_exc = StateMonitor(G_exc, ('v', 'I_stim', 'I_noise'), record=[1,2,3,4,5,6], dt=0.001*second)
    statemon_inh = StateMonitor(G_inh, ('v', 'I_noise'), record=[1,2,3,4,5,6], dt=0.001*second)
    monitors = [popmon_exc, popmon_inh, statemon_exc, statemon_inh, Mlfp]
    neuron_groups = [G_exc, G_inh, G_lfp]
    
    # Define synapses and connect neuron groups
    S_e2e=Synapses(G_exc,   G_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    S_e2i = Synapses(G_exc, G_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_e2i')
    S_i2e = Synapses(G_inh, G_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_i2e')

    S_e2e.connect(p=f'{p_e2e}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_e2i.connect(p=f'{p_e2i}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_i2e.connect(p=f'{p_i2e}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({215}*umeter)**2)))')

    S_lfp = Synapses(G_exc, G_lfp, model='''w : ohm*meter**2 (constant) # Weight in the LFP calculation
                                       v_post = w*((0.0*amp/meter**2)-Im_pre) : volt (summed)''')
    S_lfp.summed_updaters['v_post'].when = 'after_groups'  # otherwise Ic has not yet been updated for the current time step.
    S_lfp.connect()
    S_lfp.w = '(29e3 * umetre ** 2)/(4*pi*sigma)/((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2)**.5'


    synapses = [S_e2e, S_e2i, S_i2e, S_lfp]

    net = Network(neuron_groups, synapses, monitors)

    return net, synapses, monitors

In [10]:
def run_granular_simulation (net, variables, treatment_settings, monitors):
    
    print('#######################')
    print('# Starting Simulation #')
    print('#######################')
    print()
    
    total_duration = variables['duration']
    input_signal = read_input_signal(variables['input_signal_file'])
    
    time_fragment, firing_rate_threshold = treatment_settings
    popmon_exc, popmon_inh, statemon_exc, statemon_inh, Mlfp = monitors
    
    Eke = variables['Eke_baseline']
    Eki = variables['Eki_baseline']
    Eke_baseline = variables['Eke_baseline']
    Eki_baseline = variables['Eki_baseline']
    
    # Print Parameters Used
    print('Treatment Parameters:', 'time sensitivity', time_fragment, 'FR Threshold:', firing_rate_threshold)
    print()
    
    time_fragment_ms = int(time_fragment/ms)
    num_batches = int(total_duration / time_fragment)
    
    for i in tqdm(range(num_batches), desc="Running Simulation"):

        # Run the Simulation 
        net.run(time_fragment)

        if np.mean(popmon_exc.rate[-time_fragment_ms:]) > firing_rate_threshold:
            Eke = variables['Eke_treatment']
            Eki = variables['Eki_treatment']

In [11]:
def run_model_loop (variables):

    if not check_dict_lenghts(variables):
        raise ValueError('Lenghts of each variable list has be to equal!')

    variables["coord_of_electrode"] = populate_electrode_positions(variables)

    for i in range(len(variables['run_id'])):
        
        start_scope()
        defaultclock.dt = 0.001*second   

        current_variables = {key: variables[key][i] for key in variables}

        # Create folder
        run_id = current_variables["run_id"]
        os.mkdir(f'./results/{run_id}')
        write_run_settings(current_variables, run_id)
        
        print(current_variables['noise_exc'])

        # Create dynamic objects based on variables provided
        topology_exc = create_neuron_topology(current_variables['N'][0], current_variables['bounds'])
        topology_inh = create_neuron_topology(current_variables['N'][1], current_variables['bounds'])
        topologies = [topology_exc, topology_inh]
        
        stimulus_geometry_settings = [current_variables['coord_of_stimulus'], current_variables['radius_of_stimulus']]
        stimulus_mask_exc = create_spherical_mask(topology_exc, stimulus_geometry_settings)
        
        treatment_geometry_settings = [current_variables['coord_of_electrode'], current_variables['radius_of_electrode']]
        #treatment_mask_exc = create_spherical_mask(topology_exc, treatment_geometry_settings)
        #treatment_mask_inh = create_spherical_mask(topology_inh, treatment_geometry_settings)
        treatment_mask_exc = np.ones(current_variables['N'][0])
        treatment_mask_inh = np.ones(current_variables['N'][1])
        treatment_masks = [treatment_mask_exc, treatment_mask_inh]
        
        treatment_settings = [current_variables['device_sensitivity'], current_variables['firing_rate_threshold']]
        
        # Instantiate the Network
        net, synapses, monitors = prepare_network(topologies, stimulus_mask_exc, treatment_masks, current_variables)
        popmon_exc, popmon_inh, statemon_exc, statemon_inh, Mlfp = monitors
        
        # Write network statistics
        write_network_statistics(synapses, current_variables['N'], run_id)

        # Save network plots
        plot_neuron_masks (topology_exc, [stimulus_mask_exc, treatment_mask_exc], run_id)
        plot_neuron_mask (topology_exc, stimulus_mask_exc, 'red', 'stimulus', run_id)
        plot_neuron_mask (topology_exc, treatment_mask_exc, 'blue', 'treatment', run_id)

        try:
            # Run the simulation
            run_granular_simulation (net, current_variables, treatment_settings, monitors)

            # Save Firing Rate Data
            np.savetxt(f'./results/{run_id}/fr_exc.txt', popmon_exc.rate)
            np.savetxt(f'./results/{run_id}/fr_inh.txt', popmon_inh.rate)

            #Plot the Firing Rate and Noise

            plt.plot(popmon_inh.t, popmon_inh.rate, label='Inhibitory')
            plt.plot(popmon_exc.t, popmon_exc.rate, label='Excitatory')

            plt.legend()
            plt.savefig(f'./results/{run_id}/firing-rates.png', bbox_inches='tight')
            plt.close()

            plt.plot(statemon_inh.t, statemon_inh.I_noise[4]/nA, label='inh')
            plt.savefig(f'./results/{run_id}/noise_inh.png', bbox_inches='tight')
            plt.close()

            plt.plot(statemon_exc.t, statemon_exc.I_noise[4]/nA, label='exc')
            plt.savefig(f'./results/{run_id}/noise_exc.png', bbox_inches='tight')
            plt.close()

            np.savetxt(f'./results/{run_id}/LFP.txt', Mlfp.v[0]/mV)
            plot(Mlfp.t/ms, Mlfp.v[0]/mV)
            plt.savefig(f'./results/{run_id}/LFP.png', bbox_inches='tight')
            plt.close()

            
        except:
            print('Broken run.')
        
        time.sleep(1)
        

In [12]:
###############################
# Automated Runloop Variables #
###############################

copy_times = 5

# To activate epileptic mode, see "Eke_baseline" and "p" settings

variables = {
    
    # General Settings

    "run_id": ['Results 1', 'Results 2', 'Results 3', 'Results 4', 'Results 5'],
    "duration": [4000*ms]*copy_times,
    "bounds": [[600, 600, 600]]*copy_times, # [x_bound, y_bound, z_bound]
    
    # Network Settings
    
    "N": [[13500, 3375]]*copy_times, # [N_exc, N_inh]

    "Eke_baseline": [-84*mV]*copy_times,  # Set to =90 for epileptic mode
    "Eki_baseline": [-90*mV]*copy_times,

    "noise_exc": [[0.07, 0.075]*nA]*copy_times, #[0.1045, 0.104]
    "noise_inh": [[0.05, 0.08]*nA]*copy_times,

    # Normal ranges from 0.7-0.75, to activate sprouting increase the normal by 0.5
    # This will increase the average number of excitatory connections by 500
    "p": [[0.75, 0.35, 0.35, 0.0]]*copy_times, # pe2e, pe2i, pi2e, pi2i

    # Stimulus Settings

    "input_signal_file": ['sigmoid-1.0.txt']*copy_times, # files found in /stimuli/
    "coord_of_stimulus": [[300, 300, 300]]*copy_times,
    "radius_of_stimulus": [180]*copy_times,

    # Treatment Settings
    
    "device_sensitivity": [8*ms]*copy_times, # Device sensitivity - how frequently to check is firing rate is above the threshold
    "firing_rate_threshold": [5*Hz]*copy_times, # If firing rate goes above, the treatment activates

    "Eke_treatment": [-100*mV]*copy_times,
    "Eki_treatment": [-90*mV]*copy_times,

    "radius_of_electrode": [200]*copy_times,
    "distance_between_masks": [100]*copy_times,
    
}


In [13]:
run_model_loop(variables)


Radius of electrode: 200
Max distance possible between masks: 173.20508075688772
Distance selected: 100


Radius of electrode: 200
Max distance possible between masks: 173.20508075688772
Distance selected: 100


Radius of electrode: 200
Max distance possible between masks: 173.20508075688772
Distance selected: 100


Radius of electrode: 200
Max distance possible between masks: 173.20508075688772
Distance selected: 100


Radius of electrode: 200
Max distance possible between masks: 173.20508075688772
Distance selected: 100

[70. 75.] pA


cl : Command line warning D9025 : overriding '/W3' with '/w'
WARNING    Cannot use Cython, a test compilation failed: command 'C:\\Program Files (x86)\\Microsoft Visual Studio\\2022\\BuildTools\\VC\\Tools\\MSVC\\14.39.33519\\bin\\HostX86\\x64\\cl.exe' failed with exit code 2 (CompileError) [brian2.codegen.runtime.cython_rt.cython_rt.failed_compile_test]
INFO       Cannot use compiled code, falling back to the numpy code generation target. Note that this will likely be slower than using compiled code. Set the code generation to numpy manually to avoid this message:
prefs.codegen.target = "numpy" [brian2.devices.device.codegen_fallback]


_cython_magic_53cfa03f278cdb8b8876fb3be0f858b3.cpp
C:\Users\laure\.cython\brian_extensions\_cython_magic_53cfa03f278cdb8b8876fb3be0f858b3.cpp(42): fatal error C1083: Cannot open include file: 'Python.h': No such file or directory
#######################
# Starting Simulation #
#######################

Treatment Parameters: time sensitivity 8. ms FR Threshold: 5. Hz



Running Simulation:   0%|          | 0/500 [00:00<?, ?it/s]

WARNING    'glu' is an internal variable of group 'exc_group', but also exists in the run namespace with the value 1. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    inh_group's variable 'v' has NaN, very large values, or encountered an error in numerical integration. This is usually a sign that an unstable or invalid integration method was chosen. [brian2.groups.group.invalid_values]
WARNING    inh_group's variable 'm' has NaN, very large values, or encountered an error in numerical integration. This is usually a sign that an unstable or invalid integration method was chosen. [brian2.groups.group.invalid_values]
WARNING    inh_group's variable 'h' has NaN, very large values, or encountered an error in numerical integration. This is usually a sign that an unstable or invalid integration method was chosen. [brian2.groups.group.invalid_values]
WARNING    inh_group's variable 'n' has NaN, very large values, or encountered an error in 

...

Running Simulation:   0%|          | 0/500 [00:00<?, ?it/s]

[70. 75.] pA
#######################
# Starting Simulation #
#######################

Treatment Parameters: time sensitivity 8. ms FR Threshold: 5. Hz



Running Simulation:   0%|          | 0/500 [00:00<?, ?it/s]

[70. 75.] pA
#######################
# Starting Simulation #
#######################

Treatment Parameters: time sensitivity 8. ms FR Threshold: 5. Hz



Running Simulation:   0%|          | 0/500 [00:00<?, ?it/s]